In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
from opacus import PrivacyEngine

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs
from utils import averageModels, normalModels

In [57]:
class Arguments():
    def __init__(self):
        self.images = 60000
        self.clients = 5
        self.rounds = 10
        self.epochs = 5
        self.local_batches = 64
        self.lr = 0.01
        self.C = 1.0
        self.drop_rate = 0.0
        self.torch_seed = 0
        self.log_interval = 100
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [58]:
writer = open('5_experiments_mnist.csv', 'a')

In [59]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [60]:
# # Download MNIST manually using 'wget' then uncompress the file
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

In [61]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)

In [62]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
#     client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
#     client['samples'] = len(trainset_ind_list) / args.images
    client['samples'] = len(trainset_ind_list) / (len(trainset_ind_list) * args.clients)

In [63]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
global_test_dataset = datasets.MNIST('./', train=False, download=True, transform=transform)
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True)

In [64]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [65]:
def ClientUpdate(args, device, client):
    client['model'].train()
#     client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
#             data = data.send(client['hook'])
#             target = target.send(client['hook'])
            
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data)
            loss = F.nll_loss(output, target)
            loss.backward()
            client['optim'].step()
            
            if batch_idx % args.log_interval == 0:
#                 loss = loss.get() 
                print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * batch_idx / len(client['trainset']), loss))
                
#     client['model'].get() 

In [66]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    writer.write('{},'.format(100. * correct / len(test_loader.dataset)))

In [67]:
def Aggregate(all_models, sample_client):
    all_dicts = {}
    for inds, client in enumerate(all_models):
        all_dicts[client['hook'].id] = all_models[inds]['model'].state_dict()
    output_dict = copy.deepcopy(all_dicts[sample_client])
    for key in all_dicts[sample_client]:
        for val in all_dicts:
            if val == sample_client:
                continue
            output_dict[key] += all_dicts[val][key]
#         output_dict[key] /= float(len(all_dicts))
        output_dict[key] /= float(len(all_models))
    output_model = Net().to(device)
    output_model.load_state_dict(output_dict)
    return output_model

In [68]:
torch.manual_seed(args.torch_seed)
global_model = Net()
# torch.manual_seed(args.torch_seed)
# normal_avg = Net()
# torch.manual_seed(args.torch_seed)
# fake_model = Net()

for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)
#     print('images {} batches {} sample size {}'.format(args.images, args.local_batches, len(client['trainset'])))
#     print(args.local_batches/len(client['trainset']))
    client['pengine'] = PrivacyEngine(
                                       client['model'],
                                       batch_size=args.local_batches,
                                       sample_size=len(client['trainset']),
                                       sample_rate=0.01,
                                       alphas=range(2,32),
                                       noise_multiplier=1.3,
                                       max_grad_norm=1.0,
                                    )
    client['pengine'].attach(client['optim'])

start_time = time.time()

for fed_round in range(args.rounds):
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))
    
    # number of selected clients
    m = int(max(args.C * args.clients, 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
    np.random.seed(fed_round)
    active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
    active_clients = [clients[i] for i in active_clients_inds]
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
    
#     # Testing 
#     for client in active_clients:
#         test(args, client['model'], device, client['testset'], client['hook'].id)
    
    # Averaging 
    global_model = averageModels(global_model, active_clients)
#     normal_avg = normalModels(normal_avg, active_clients)
#     fake_model = Aggregate(active_clients, 'client1')
    
    # Testing the average model
    test(args, global_model, device, global_test_loader, 'Global')
#     test(args, normal_avg, device, global_test_loader, 'Global')
#     test(args, fake_model, device, global_test_loader, 'Global')
    
#     break
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())

writer.write('\n')
end_time = time.time() 
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
writer.write('{:0>2}:{:0>2}:{:05.2f}'.format(int(hours),int(minutes),seconds))
writer.write('\n')
writer.close()

if (args.save_model):
    torch.save(global_model.state_dict(), "FedAvg.pt")

/Users/gharibi/opt/anaconda3/envs/iswc/lib/python3.7/site-packages/opacus/privacy_engine.py:646: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/Users/gharibi/opt/anaconda3/envs/iswc/lib/python3.7/site-packages/opacus/privacy_engine.py:230: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "
/Users/gharibi/opt/anaconda3/envs/iswc/lib/python3.7/site-packages/opacus/privacy_engine.py:646: UserWarning: A ``sample_rate`` has been provided.Thus, the provided ``batch_size``and ``sample_size`` will be ignored.
  "A ``sample_rate`` has been provided."
/Users/gharibi/opt/anaconda3/envs/iswc/lib/python3.7/site-packages/opacus

Model client2 Train Epoch: 1 [0/12032 (0%)]	Loss: 2.289522
Model client2 Train Epoch: 1 [6400/12032 (53%)]	Loss: 2.237266
Model client2 Train Epoch: 2 [0/12032 (0%)]	Loss: 2.229522
Model client2 Train Epoch: 2 [6400/12032 (53%)]	Loss: 2.124352
Model client2 Train Epoch: 3 [0/12032 (0%)]	Loss: 2.078828
Model client2 Train Epoch: 3 [6400/12032 (53%)]	Loss: 2.017442
Model client2 Train Epoch: 4 [0/12032 (0%)]	Loss: 1.850002
Model client2 Train Epoch: 4 [6400/12032 (53%)]	Loss: 1.742391
Model client2 Train Epoch: 5 [0/12032 (0%)]	Loss: 1.584490
Model client2 Train Epoch: 5 [6400/12032 (53%)]	Loss: 1.507055
Model client3 Train Epoch: 1 [0/12032 (0%)]	Loss: 2.317182
Model client3 Train Epoch: 1 [6400/12032 (53%)]	Loss: 2.263371
Model client3 Train Epoch: 2 [0/12032 (0%)]	Loss: 2.226937
Model client3 Train Epoch: 2 [6400/12032 (53%)]	Loss: 2.147763
Model client3 Train Epoch: 3 [0/12032 (0%)]	Loss: 2.038217
Model client3 Train Epoch: 3 [6400/12032 (53%)]	Loss: 1.983284
Model client3 Train Epoc

Model client4 Train Epoch: 2 [0/12032 (0%)]	Loss: 0.790236
Model client4 Train Epoch: 2 [6400/12032 (53%)]	Loss: 0.679963
Model client4 Train Epoch: 3 [0/12032 (0%)]	Loss: 0.603696
Model client4 Train Epoch: 3 [6400/12032 (53%)]	Loss: 0.592382
Model client4 Train Epoch: 4 [0/12032 (0%)]	Loss: 0.494817
Model client4 Train Epoch: 4 [6400/12032 (53%)]	Loss: 0.519337
Model client4 Train Epoch: 5 [0/12032 (0%)]	Loss: 0.593930
Model client4 Train Epoch: 5 [6400/12032 (53%)]	Loss: 0.659470
Model client3 Train Epoch: 1 [0/12032 (0%)]	Loss: 0.921196
Model client3 Train Epoch: 1 [6400/12032 (53%)]	Loss: 0.626134
Model client3 Train Epoch: 2 [0/12032 (0%)]	Loss: 0.839058
Model client3 Train Epoch: 2 [6400/12032 (53%)]	Loss: 0.680651
Model client3 Train Epoch: 3 [0/12032 (0%)]	Loss: 0.628497
Model client3 Train Epoch: 3 [6400/12032 (53%)]	Loss: 0.528559
Model client3 Train Epoch: 4 [0/12032 (0%)]	Loss: 0.438786
Model client3 Train Epoch: 4 [6400/12032 (53%)]	Loss: 0.517773
Model client3 Train Epoc

Model client5 Train Epoch: 2 [6400/12032 (53%)]	Loss: 0.539968
Model client5 Train Epoch: 3 [0/12032 (0%)]	Loss: 0.514219
Model client5 Train Epoch: 3 [6400/12032 (53%)]	Loss: 0.550243
Model client5 Train Epoch: 4 [0/12032 (0%)]	Loss: 0.678173
Model client5 Train Epoch: 4 [6400/12032 (53%)]	Loss: 0.468174
Model client5 Train Epoch: 5 [0/12032 (0%)]	Loss: 0.553576
Model client5 Train Epoch: 5 [6400/12032 (53%)]	Loss: 0.444219
Model client1 Train Epoch: 1 [0/12032 (0%)]	Loss: 0.413919
Model client1 Train Epoch: 1 [6400/12032 (53%)]	Loss: 0.466944
Model client1 Train Epoch: 2 [0/12032 (0%)]	Loss: 0.498575
Model client1 Train Epoch: 2 [6400/12032 (53%)]	Loss: 0.578850
Model client1 Train Epoch: 3 [0/12032 (0%)]	Loss: 0.714761
Model client1 Train Epoch: 3 [6400/12032 (53%)]	Loss: 0.682675
Model client1 Train Epoch: 4 [0/12032 (0%)]	Loss: 0.457010
Model client1 Train Epoch: 4 [6400/12032 (53%)]	Loss: 0.583525
Model client1 Train Epoch: 5 [0/12032 (0%)]	Loss: 0.314749
Model client1 Train Epoc

Model client5 Train Epoch: 3 [6400/12032 (53%)]	Loss: 0.596851
Model client5 Train Epoch: 4 [0/12032 (0%)]	Loss: 0.840320
Model client5 Train Epoch: 4 [6400/12032 (53%)]	Loss: 0.303875
Model client5 Train Epoch: 5 [0/12032 (0%)]	Loss: 0.666055
Model client5 Train Epoch: 5 [6400/12032 (53%)]	Loss: 0.746467

Test set: Average loss for Global model: 0.4594, Accuracy: 8868/10000 (89%)

Model client4 Train Epoch: 1 [0/12032 (0%)]	Loss: 0.447988
Model client4 Train Epoch: 1 [6400/12032 (53%)]	Loss: 1.153986
Model client4 Train Epoch: 2 [0/12032 (0%)]	Loss: 0.801482
Model client4 Train Epoch: 2 [6400/12032 (53%)]	Loss: 0.193791
Model client4 Train Epoch: 3 [0/12032 (0%)]	Loss: 0.345811
Model client4 Train Epoch: 3 [6400/12032 (53%)]	Loss: 0.307373
Model client4 Train Epoch: 4 [0/12032 (0%)]	Loss: 0.435460
Model client4 Train Epoch: 4 [6400/12032 (53%)]	Loss: 0.770697
Model client4 Train Epoch: 5 [0/12032 (0%)]	Loss: 0.572387
Model client4 Train Epoch: 5 [6400/12032 (53%)]	Loss: 0.552502
Model

In [69]:
clients[9]['trainset']

IndexError: list index out of range